In [395]:
import pandas as pd
import numpy as np
import joblib
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

In [396]:
df = pd.read_csv("../data/train.csv")

In [397]:
df = df.drop(["Unnamed: 0"],axis=1)
df = df.set_index("id")
label = df["stroke"]
df = df.drop(["stroke"],axis=1)

In [398]:
df.head()

,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status
id,,,,,,,,,,
9046,Male,67.0,0,1,Yes,Private,Urban,228.69,36.6,formerly smoked
51676,Female,61.0,0,0,Yes,Self-employed,Rural,202.21,NaN,never smoked
31112,Male,80.0,0,1,Yes,Private,Rural,105.92,32.5,never smoked
60182,Female,49.0,0,0,Yes,Private,Urban,171.23,34.4,smokes
1665,Female,79.0,1,0,Yes,Self-employed,Rural,174.12,24.0,never smoked


In [399]:
df.isnull().sum()

gender                 0
age                    0
hypertension           0
heart_disease          0
ever_married           0
work_type              0
Residence_type         0
avg_glucose_level      0
bmi                  177
smoking_status         0
dtype: int64

In [400]:
df["bmi"].mode()

0    26.7
1    27.7
Name: bmi, dtype: float64

In [401]:
# Find mode value 
mode_1 = df["bmi"].mode()[0]
joblib.dump(mode_1,"../model_clf/mode")


['../model_clf/mode']

In [402]:
mode_1 = df["bmi"].mode()[1]
joblib.dump(mode_1,"../model_tree/mode")

['../model_tree/mode']

In [403]:
def missing_values(df):
    
    mode = joblib.load("../model_clf/mode")
    df = df.fillna(mode)
    return df
    
    

In [404]:
def fit_one_hot(df):
    encoder = OneHotEncoder(handle_unknown='ignore',sparse_output=False)
    encoder.fit(df[["gender","ever_married","work_type","Residence_type"]])
    joblib.dump(encoder,"../model_clf/encoder")

In [405]:
def transform_one_hot(df):
    encoder = joblib.load("../model_clf/encoder")
    result = encoder.transform(df[["gender","ever_married","work_type","Residence_type"]])
    df = df.drop(["gender","work_type","Residence_type","ever_married"],axis=1)
    df[encoder.get_feature_names_out()] = result
    return df

In [406]:
def fit_ordinal(df):
    encoder = OrdinalEncoder(handle_unknown="use_encoded_value",unknown_value=4)
    encoder.fit(df[["smoking_status"]])
    joblib.dump(encoder,"../model_clf/encoder_ordinal")

In [407]:
def transform_ordinal(df):
    encoder = joblib.load("../model_clf/encoder_ordinal")
    result = encoder.transform(df[["smoking_status"]])
    df[encoder.get_feature_names_out()] = result
    return df

In [408]:
def min_max_scaler_fit(df):
    scaler = MinMaxScaler()
    scaler.fit(df)
    joblib.dump(scaler,"../model_clf/scaler")

In [409]:
def min_max_scaler_transform(df):
    scaler = joblib.load("../model_clf/scaler")
    result = scaler.transform(df)
    df.iloc[:,:] = result
    return df

In [410]:
def train_model(df, label):
    x_train, x_test, y_train, y_test = split_test_train(df,label)
    clf = LogisticRegression()
    clf.fit(x_train,y_train)
    joblib.dump(clf,"../model_clf/clf")
    return clf.score(x_test,y_test)
    

In [411]:
def split_test_train(df,label):
    X = df
    Y = label
    x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state=42)
    return x_train, x_test, y_train, y_test
    

In [412]:
def pipeline(df,label):
    df = missing_values(df)
    
    fit_one_hot(df)
    df = transform_one_hot(df)
    
    fit_ordinal(df)
    df = transform_ordinal(df)
    
    min_max_scaler_fit(df)
    df = min_max_scaler_transform(df)
    score = train_model(df, label)
    return score
    
    
    

In [413]:
pipeline(df, label)

/tmp/ipykernel_80266/1189991753.py:4: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df.iloc[:,:] = result


0.9274074074074075